<a href="https://colab.research.google.com/github/Nixtla/hierarchicalforecast/blob/main/examples/AustralianPrisonPopulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Forecasting Australian prison population

In this notebook we will explain how to produce coherent forecast for the Australian prison population across diferent groups, replicating the results of the book [Forecasting: Principles and Practice](https://otexts.com/fpp3/prison.html).

In [ ]:
%%capture
!pip install hierarchicalforecast
!pip install -U statsforecast numba statsmodels

In [ ]:
import numpy as np
import pandas as pd

#obtain hierarchical reconciliation methods and evaluation
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.evaluation import HierarchicalEvaluation
from hierarchicalforecast.methods import BottomUp, MinTrace
from hierarchicalforecast.utils import aggregate
# compute base forecast no coherent
from statsforecast.core import StatsForecast
from statsforecast.models import ets

## Aggregate bottom time series

The dataset only contains the time series at the lowest level, so we need to create the time series for all hierarchies.

In [ ]:
Y_df = pd.read_csv('https://OTexts.com/fpp3/extrafiles/prison_population.csv')
Y_df = Y_df.rename({'Count': 'y', 'Date': 'ds'}, axis=1)
Y_df.insert(0, 'Country', 'Australia')
Y_df = Y_df[['Country', 'State', 'Gender', 'Legal', 'Indigenous', 'ds', 'y']]
Y_df['ds'] = pd.to_datetime(Y_df['ds'])
Y_df.head()

,Country,State,Gender,Legal,Indigenous,ds,y
0,Australia,ACT,Female,Remanded,ATSI,2005-03-01,0
1,Australia,ACT,Female,Remanded,Non-ATSI,2005-03-01,2
2,Australia,ACT,Female,Sentenced,ATSI,2005-03-01,0
3,Australia,ACT,Female,Sentenced,Non-ATSI,2005-03-01,5
4,Australia,ACT,Male,Remanded,ATSI,2005-03-01,7


The dataset can be grouped in the following grouped structure.

In [ ]:
hiers = [
    ['Country'],
    ['Country', 'State'], 
    ['Country', 'Gender'], 
    ['Country', 'Legal'], 
    ['Country', 'State', 'Gender', 'Legal']
]

Using the `aggregate` function from `HierarchicalForecast` we can get the full set of time series.

In [ ]:
Y_df, S, tags = aggregate(Y_df, hiers, lambda x: np.sum(x) / 1e3)
Y_df = Y_df.reset_index()

In [ ]:
Y_df.head()

,unique_id,ds,y
0,Australia,2005-03-01,24.296
1,Australia,2005-06-01,24.643
2,Australia,2005-09-01,24.511
3,Australia,2005-12-01,24.393
4,Australia,2006-03-01,24.524


In [ ]:
S.iloc[:5, :5]

,Australia/ACT/Female/Remanded,Australia/ACT/Female/Sentenced,Australia/ACT/Male/Remanded,Australia/ACT/Male/Sentenced,Australia/NSW/Female/Remanded
Australia,1.0,1.0,1.0,1.0,1.0
Australia/ACT,1.0,1.0,1.0,1.0,0.0
Australia/NSW,0.0,0.0,0.0,0.0,1.0
Australia/NT,0.0,0.0,0.0,0.0,0.0
Australia/QLD,0.0,0.0,0.0,0.0,0.0


In [ ]:
tags

{'Country': array(['Australia'], dtype=object),
 'Country/State': array(['Australia/ACT', 'Australia/NSW', 'Australia/NT', 'Australia/QLD',
        'Australia/SA', 'Australia/TAS', 'Australia/VIC', 'Australia/WA'],
       dtype=object),
 'Country/Gender': array(['Australia/Female', 'Australia/Male'], dtype=object),
 'Country/Legal': array(['Australia/Remanded', 'Australia/Sentenced'], dtype=object),
 'Country/State/Gender/Legal': array(['Australia/ACT/Female/Remanded', 'Australia/ACT/Female/Sentenced',
        'Australia/ACT/Male/Remanded', 'Australia/ACT/Male/Sentenced',
        'Australia/NSW/Female/Remanded', 'Australia/NSW/Female/Sentenced',
        'Australia/NSW/Male/Remanded', 'Australia/NSW/Male/Sentenced',
        'Australia/NT/Female/Remanded', 'Australia/NT/Female/Sentenced',
        'Australia/NT/Male/Remanded', 'Australia/NT/Male/Sentenced',
        'Australia/QLD/Female/Remanded', 'Australia/QLD/Female/Sentenced',
        'Australia/QLD/Male/Remanded', 'Australia/QLD/Male

### Split Train/Test sets

We use the final two years (8 quarters) as test set.

In [ ]:
Y_df_test = Y_df.groupby('unique_id').tail(8)
Y_df_train = Y_df.drop(Y_df_test.index)

In [ ]:
Y_df_test = Y_df_test.set_index('unique_id')
Y_df_train = Y_df_train.set_index('unique_id')

## Computing base forecasts

The following cell computes the **base forecasts** for each time series in `Y_df` using the `auto_arima` and `naive` models. Observe that `Y_hat_df` contains the forecasts but they are not coherent.

In [ ]:
fcst = StatsForecast(df=Y_df_train, 
                     models=[(ets, 4, 'ZMZ')], 
                     freq='QS', n_jobs=-1)
Y_hat_df = fcst.forecast(h=8, fitted=True)
Y_fitted_df = fcst.forecast_fitted_values()

## Reconcile forecasts

The following cell makes the previous forecasts coherent using the `HierarchicalReconciliation` class. Since the hierarchy structure is not strict, we can't use methods such as `TopDown` or `MiddleOut`. In this example we use `BottomUp` and `MinTrace`.

In [ ]:
reconcilers = [
    BottomUp(),
    MinTrace(method='mint_shrink')
]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)
Y_rec_df = hrec.reconcile(Y_hat_df, Y_fitted_df, S, tags)

The dataframe `Y_rec_df` contains the reconciled forecasts.

In [ ]:
Y_rec_df.head()

,ds,ets_season_length-4_model-ZMZ,ets_season_length-4_model-ZMZ/BottomUp,ets_season_length-4_model-ZMZ/MinTrace_method-mint_shrink
unique_id,,,,
Australia,2015-01-01,34.799496,34.933872,34.963669
Australia,2015-04-01,35.192638,35.473522,35.521146
Australia,2015-07-01,35.188217,35.687302,35.630352
Australia,2015-10-01,35.888626,36.010601,36.147547
Australia,2016-01-01,36.045437,36.400002,36.492036


## Evaluation 

The `HierarchicalForecast` package includes the `HierarchicalEvaluation` class to evaluate the different hierarchies and also is capable of compute scaled metrics compared to a benchmark model.

In [ ]:
def mase(y, y_hat, y_insample, seasonality=4):
    errors = np.mean(np.abs(y - y_hat), axis=1)
    scale = np.mean(np.abs(y_insample[:, seasonality:] - y_insample[:, :-seasonality]), axis=1)
    return np.mean(errors / scale)

eval_tags = {}
eval_tags['Total'] = tags['Country']
eval_tags['State'] = tags['Country/State']
eval_tags['Legal status'] = tags['Country/Legal']
eval_tags['Gender'] = tags['Country/Gender']
eval_tags['Bottom'] = tags['Country/State/Gender/Legal']
eval_tags['All series'] = np.concatenate(list(tags.values()))

evaluator = HierarchicalEvaluation(evaluators=[mase])
evaluation = evaluator.evaluate(
    Y_h=Y_rec_df, Y_test=Y_df_test, 
    tags=eval_tags,
    Y_df=Y_df_train    
)
evaluation = evaluation.reset_index().drop(columns='metric').drop(0).set_index('level')
evaluation.columns = ['Base', 'BottomUp', 'MinTrace(mint_shrink)']
evaluation.applymap('{:.2f}'.format)

,Base,BottomUp,MinTrace(mint_shrink)
level,,,
Total,1.36,1.02,0.96
State,1.66,1.57,1.60
Legal status,2.40,2.50,2.42
Gender,1.08,0.81,0.80
Bottom,2.17,2.17,2.18
All series,2.03,2.00,2.00


### Comparison fable

Observe that we can recover the results reported by the [Forecasting: Principles and Practice](https://otexts.com/fpp3/prison.html) book. The original results were calculated using the R package [fable](https://github.com/tidyverts/fable).

![image](./AustralianPrisonPopulation-results-fable.png)